# RESOLUTION DE JEUX - TP1 -HASHCODE
## SOUMISSION - FORME MATRICIELLE -> CHOIX FINAL 

**meilleure soumission : 1387666 pts**

* Chargment des variables de départ : 
    - Le nombre de vidéos : nb_vid_total
    - Le nombre de endpoints : nb_endPoint_total
    - Le nombre de descriptions de requests : nb_requestDesc_total
    - Le nombre de caches : nb_cache_total
    - La taille des caches : mem_sto_total
* Création de deux matrices :
    - Une contenant le nombre de requests de chaque vidéo pour chaque endpoint " Matrix_endpoint_video_request " 
    - Une contenant le temps de latence entre chaque cache et chaque endpoint " Matrix_endpoint_cache_latency "
* Création de deux listes : 
    - Liste des tailles de vidéos, l'index de position est le numero de video "Size_vid"
    - Liste des tailles de caches, l'index de position est le numero de cache "caches_size"
* Itération : 
    - Pour chaque vidéo **i_vid** de **0 à nb_vid_total**
    - Pour chaque endpoint **j_endp**, dans l'ordre décroissant du nombre total de request à ce endpoint
    - Si on a des requests pour la vidéo **i_vid** à ce endpoint **j_endp**,
    - Et qu'il y a au moins un cache relié à ce endp
    - On prend les caches **i_cache** dans l'ordre croissant de leur taille **caches_size** disponible (pour remplir ceux qui ont le moins de place en premier si possible)
    - Si il y a de la place pour la vidéo i_vid de **Size_vid**, dans le **i_cache** on met la video **i_vid** 
    - et on diminue la taille du **i_cache** dans **caches_size** en enlevant **Size_vid**

In [ ]:
import sys
import numpy as np

# ------ décommenter pour tester en local ------- #
#filin = open("me_at_the_zoo.in", "r")   # ok 
#filin = open("kittens.in.txt", "r")
#filin = open("trending_today.in", "r")
#filin = open("videos_worth_spreading.in", "r")
#filin = open("input.txt", "r")  # ok
#filin = open("input2.txt", "r") # ok
#filin = open("input3.txt", "r") #ok
#lignes= filin.readlines()
#lignes = [ligne.strip() for ligne in lignes]
# ----------------------------------------------- #


lignes= sys.stdin.readlines()
lignes = [ligne.strip() for ligne in lignes]
# ------------- Stockage des données de départ ---------------- #
nb_vid_total, nb_endPoint_total, nb_requestDesc_total, nb_cache_total, mem_sto_total = [int(i) for i in lignes[0].split(" ")]

# taille des vidéos
Size_vid = [int(i) for i in lignes[1].split(" ")]
#taille des caches
caches_size = [int(mem_sto_total) for i in range(nb_cache_total)] #liste de taille de chaque cache

# ----------- MATRICE CACHE_ENDPOINTS_LATENCY ----------------- #

Matrix_endpoint_cache_latency = np.zeros((nb_endPoint_total,nb_cache_total))
lign = 2 

for i in range(nb_endPoint_total):
    endpoint_nb = i  
    dc_latency, nb_caches = [int(i) for i in lignes[lign].split(" ")] 
    if nb_caches != 0: 
        for cache_col in range(1,nb_caches+1):
            cache_name, cache_latency = [int(i) for i in lignes[lign+cache_col].split(" ")]
            Matrix_endpoint_cache_latency[i,int(cache_name)] = cache_latency
    lign = lign + nb_caches +1

# --------------- MATRICE ENDPOINT_VIDEOS_REQUEST ------------- #

Matrix_endpoint_video_request = np.zeros((nb_endPoint_total,nb_vid_total))
lign = lign-1

for index in range(lign+1, len(lignes)): 
    vid_ID, endpoint_ID, nb_request = [int(i) for i in lignes[index].split(" ")] 
    Matrix_endpoint_video_request[int(endpoint_ID),int(vid_ID)] = nb_request

# ------------------------ SOLUCE  ---------------------------- #
soluce = []
realised_videos = []
def find_soluce() :
    endpoint_requests = np.sum(Matrix_endpoint_video_request, axis = 1) #nb de request totale par endpoint
    for i_vid in range(nb_vid_total):
        for j_endp in list(reversed(endpoint_requests.argsort())): # itération par endpoint ayant le plus de request liée jusuq'à celui qui en a le moins
            if int(Matrix_endpoint_video_request[j_endp,i_vid]) != 0 and sum(Matrix_endpoint_cache_latency[j_endp,]==0) != nb_cache_total: #s'il y a des requests et et cache relié au endpoint
                index_cache = np.argsort(caches_size) #on prend dans l'ordre des caches les plus petits pour les remplir
                for i_cache in index_cache :
                    if Matrix_endpoint_cache_latency[j_endp,][i_cache] != 0 :
                        if int(Size_vid[i_vid]) <= int(caches_size[i_cache]) :
                            soluce.append(["cache:",i_cache ,"vid: ",i_vid])
                            caches_size[i_cache] = int(caches_size[i_cache]) - int(Size_vid[i_vid])
                            realised_videos.append(i_vid)
                            break
                break

# ------------------------ OUTPUT  ---------------------------- #   

#appel de la fonction
find_soluce()

#affichage
cache_uses = []
for i in range(len(soluce)):
    cache_uses.append(soluce[i][1])


res = []
for i in np.unique(cache_uses):
    vid_list = []
    vid_list.append(i) #le cache
    for j in range(len(soluce)):
        if soluce[j][1]== i :
            vid_list.append(soluce[j][3])
    res.append(vid_list)
                    


print(len(np.unique(cache_uses)))
for i in res :
    print(str(i).strip('[]').replace(",",""))